<a href="https://colab.research.google.com/github/cshengmin/Stockbuylowsellhigh/blob/main/new_with_error_handling_and_two_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 📌 FULL COMBINED SCRIPT FOR GOOGLE COLAB

# Step 1: Install required libraries
!pip install yfinance yahooquery --quiet

# Step 2: Execute the full pipeline
import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

# ✅ USER INPUT: Add your tickers
# stock_tickers = ["AAPL", "MSFT", "GOOGL", "NVDA", "AMZN"]  # Add your own tickers
# etf_tickers = ["QQQ", "IVV", "SPY"]  # Add your ETF tickers

# stock_tickers = [
#     "NVDA", "META", "MU", "AMZN", "CRM", "MSFT", "GOOGL", "AVGO", "NOW",
#     "AAPL", "WMT", "HD", "PLTR", "TSM", "APP", "HIMS", "CRWD"
# ]

etf_tickers = [
    "QQQ", "QQQM", "IVV", "SPLG", "TQQQ", "SPYU", "SCHG", "SPY", "IAU","SOXL"
]

stock_tickers = [
    "AAPL", "AMD", "AMZN", "APP", "ARM", "ASML", "AVGO", "BDX", "C", "CHWY",
    "COIN", "CRM", "CRWD", "CVX", "CYBR", "DAL", "DIS", "DJT", "ENTG", "EVER",
    "GE", "GOOGL", "GS", "HD", "HES", "HIMS", "HOOD", "ICHR", "INTC", "JPM",
    "META", "MSFT", "MU", "NOW", "NVDA", "PLTR", "ROKU", "RXRX", "SERV", "SOFI",
    "SOUN", "SPOT", "SU", "TSLA", "TSM", "UBER", "UNH", "VKTX", "VRSN", "VST",
    "WMT", "WRD"
]

# #earning dates 5/5
# stock_tickers = [
#     "PLTR", "AMD", "LMND", "RIVN", "SMCI", "NVO", "UBER", "DIS",
#     "ARM", "SHOP", "VTRS", "NET", "COIN", "PINS"
# ]

# etf_tickers = [
#     "ICOP", "ILIT", "ITOT", "IVV", "QQQ", "QQQM", "REZ", "SCHF",
#     "SCHG", "SPLG", "VUG", "XLC", "XLF", "XLG"
# ]



# # ✅ INPUT: Your tickers
# stock_tickers = ["AAPL", "MSFT", "GOOGL", "NVDA", "AMZN"]
# etf_tickers = ["QQQ", "IVV", "SPY"]

all_tickers = stock_tickers + etf_tickers


# 📦 Utilities
def chunk_list(lst, size=10):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

def safe_download(tickers, start, end, retries=2, delay=4):
    for attempt in range(retries):
        try:
            df = yf.download(tickers, start=start, end=end, group_by="ticker", auto_adjust=True, progress=False)
            if not df.empty:
                return df
        except Exception as e:
            print(f"⚠️ Retry {attempt + 1}/{retries} failed for {tickers}: {e}")
        time.sleep(delay * (attempt + 1))  # exponential backoff
    print(f"❌ Skipped after {retries} failed attempts: {tickers}")
    return pd.DataFrame()

# 📅 Dates
dates_to_pull = ["2025-03-19", "2025-03-26", "2025-04-04"]

# 🔁 Historical prices
hist_prices = {}
for batch in chunk_list(all_tickers):
    data = safe_download(batch, "2025-03-18", "2025-04-05")
    if isinstance(data.columns, pd.MultiIndex):
        for t in batch:
            if t in data.columns.levels[0]:
                hist_prices[t] = data[t]
    else:
        t = batch[0]
        hist_prices[t] = data
    time.sleep(2)

def get_price_on_date(ticker, date):
    try:
        return round(hist_prices[ticker]["Close"].loc[date], 2)
    except:
        return np.nan

price_snapshots = {t: " \\ ".join([str(get_price_on_date(t, d)) for d in dates_to_pull]) for t in all_tickers}

# 🔁 10-day Moving Avg
dma_10, hist = {}, {}
end = datetime.today()
start = end - timedelta(days=15)

for batch in chunk_list(all_tickers):
    data = safe_download(batch, start, end)
    if isinstance(data.columns, pd.MultiIndex):
        for t in batch:
            if t in data.columns.levels[0]:
                hist[t] = data[t]
    else:
        t = batch[0]
        hist[t] = data
    time.sleep(2)

for t in all_tickers:
    try:
        if t in hist and not hist[t]["Close"].isna().all():
            dma = hist[t]["Close"].rolling(10).mean()
            dma_10[t] = dma.iloc[-1] if not dma.empty else np.nan
        else:
            dma_10[t] = np.nan
    except:
        dma_10[t] = np.nan

# 📊 Current prices and 52W range
current_prices, week_52 = {}, {}
for batch in chunk_list(all_tickers):
    try:
        yf_data = yf.Tickers(" ".join(batch))
        for t in batch:
            info = yf_data.tickers[t].info
            current_prices[t] = info.get("regularMarketPrice", np.nan)
            week_52[t] = f'{info.get("fiftyTwoWeekLow", "N/A")} - {info.get("fiftyTwoWeekHigh", "N/A")}'
    except Exception as e:
        print(f"❌ Failed to load summary data for {batch}: {e}")
        for t in batch:
            current_prices[t] = np.nan
            week_52[t] = "N/A"
    time.sleep(2)

# 🔎 Analyst rating and target
analyst_rating, target_price = {}, {}
for batch in chunk_list(all_tickers):
    try:
        yq = Ticker(batch)
        fd = yq.financial_data
        for t in batch:
            try:
                rating = fd[t]["recommendationMean"]
                analyst_rating[t] = (
                    "Strong Buy" if rating <= 2 else
                    "Buy" if rating <= 2.5 else
                    "Hold" if rating <= 3 else
                    "Sell" if rating <= 4 else "N/A"
                )
            except:
                analyst_rating[t] = "N/A"
            try:
                target_price[t] = fd[t]["targetMeanPrice"]
            except:
                target_price[t] = np.nan
    except Exception as e:
        print(f"❌ Analyst/target price failed for {batch}: {e}")
        for t in batch:
            analyst_rating[t] = "N/A"
            target_price[t] = np.nan
    time.sleep(2)

# 📉 Discount/Premium calc
discount_premium = {}
for t in all_tickers:
    diffs = []
    for d in dates_to_pull:
        try:
            past_price = hist_prices[t]["Close"].loc[d]
            curr_price = current_prices[t]
            if pd.notna(past_price) and pd.notna(curr_price):
                pct = ((curr_price - past_price) / past_price) * 100
                sign = "+" if pct >= 0 else "-"
                diffs.append(f"{sign}{abs(pct):.2f}%")
            else:
                diffs.append("N/A")
        except:
            diffs.append("N/A")
    discount_premium[t] = " \\ ".join(diffs)

# 📋 Final DataFrame
df = pd.DataFrame({
    "Ticker": all_tickers,
    "Current Price ($)": [current_prices.get(t, np.nan) for t in all_tickers],
    "Historic Prices (3/19 \\ 3/26 \\ 4/4)": [price_snapshots.get(t, "N/A") for t in all_tickers],
    "Discount/Premium (3/19 \\ 3/26 \\ 4/4)": [discount_premium.get(t, "N/A") for t in all_tickers],
    "10-Day Moving Avg ($)": [round(dma_10[t], 2) if not pd.isna(dma_10[t]) else None for t in all_tickers],
    "52-Week Range": [week_52.get(t, "N/A") for t in all_tickers],
    "Analyst Rating": [analyst_rating.get(t, "N/A") for t in all_tickers],
    "Target Price ($)": [target_price.get(t, np.nan) for t in all_tickers]
})

# ✂️ Split output
stock_df = df[df["Ticker"].isin(stock_tickers)].reset_index(drop=True)
etf_df = df[df["Ticker"].isin(etf_tickers)].reset_index(drop=True)

# 🖥️ Display
print("📊 Portfolio Snapshot —", datetime.today().strftime('%Y-%m-%d'))
print("\n🟦 STOCKS:")
display(stock_df)
print("\n🟧 ETFs:")
display(etf_df)

# 🚨 Report failed
failed = [t for t in all_tickers if t not in hist_prices]
if failed:
    print("\n❌ These tickers failed to download historical prices:")
    print(failed)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 27.0 MB/s eta 0:00:00


[*********************100%***********************]  10 of 10 completed
ERROR:yfinance:
10 Failed downloads:
ERROR:yfinance:['AVGO', 'CHWY', 'BDX', 'AAPL', 'APP', 'C', 'AMZN', 'AMD', 'ASML', 'ARM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  9 of 10 completed
ERROR:yfinance:
10 Failed downloads:
ERROR:yfinance:['DIS', 'COIN', 'DAL', 'CYBR', 'DJT', 'EVER', 'CVX', 'CRM', 'ENTG', 'CRWD']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  10 of 10 completed
ERROR:yfinance:
10 Failed downloads:
ERROR:yfinance:['INTC', 'GOOGL', 'GE', 'HD', 'JPM', 'HOOD', 'HIMS', 'HES', 'ICHR', 'GS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  10 of 10 completed
ERROR:yfinance:
10 Failed downloads:
ERROR:yfinance:['ROKU', 'SOFI', 'SERV', 'NVDA', 'RXRX', 'NOW', 'MSFT', 'PLTR', 'M

KeyboardInterrupt: 